In [2]:
#DATASET SINTÉTICO 3

import numpy as np
import pandas as pd


np.random.seed(42)

#Numero de muestras

n_samples = 1000

# Variables exógenas  (no dependen de ninguna otra)

Z = np.random.normal(0,1,n_samples)  # confusor

X1 = 2*Z + np.random.normal(0,1,n_samples)
X2 = 0.67*Z + np.random.uniform(-2, 2, n_samples)
X3 = np.random.choice([0, 1], size= n_samples, p=[0.6, 0.4])

#Variables derivadas no lineales
Y1 = np.sin(np.pi * X1) + np.random.normal(0, 0.2, n_samples)


# Variables categóricas derivadas

def gen_Y2(X3, Y1):
    if X3 == 0 and Y1 < 0.5:
        return np.random.choice([0, 1], p=[0.7, 0.3])
    elif X3 == 1 and  X3 >= 0.5:
        return np.random.choice([0, 1], p=[0.2, 0.8])
    else:
        return np.random.choice([0, 1], p=[0.4, 0.6])
Y2 = np.array([gen_Y2(x3, y1) for x3, y1 in zip(X3, Y1)])

def gen_Y3(Y1, Y2): # Label
    if Y1 < 0.7 and Y2 == 0:
        return np.random.choice([0, 1], p=[0.5, 0.5])
    elif Y1 >=0.7 and  Y2 == 1:
        return np.random.choice([0, 1], p=[0.65, 0.35])
    else:
        return np.random.choice([0, 1], p=[0.1, 0.9])
Y3 = np.array([gen_Y3(y1, y2) for y1, y2 in zip(Y1, Y2)])

# Variables independientes o con ruido puro
Noise1 = np.random.normal(0, 1, n_samples)
Noise2 = np.random.normal(10, 1, n_samples)


df_three = pd.DataFrame({
    "Z": Z, "X1": X1, "X2": X2, "X3": X3,
    "Y1": Y1, "Y2": Y2, "Y3": Y3, "Noise1": Noise1, 
    "Noise2": Noise2
})

df_three.head()

,Z,X1,X2,X3,Y1,Y2,Y3,Noise1,Noise2
0,0.496714,2.392784,-0.038776,0,1.157483,1,0,-0.161955,11.085222
1,-0.138264,0.648105,-1.828598,0,1.014518,0,1,0.438413,10.038555
2,0.647689,1.355007,-0.170767,1,-0.437310,1,1,-0.883999,9.516341
3,1.523030,2.399123,-0.535578,0,0.654313,1,1,0.132497,9.753276
4,-0.234153,0.229917,1.076058,0,0.912962,1,1,-0.381489,9.831580


In [3]:
# Normalizamos las variables continuas

from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
numerical_cols = ["Z","X1", "X2", "Y1", "Noise1", "Noise2" ]
df_three[numerical_cols] = scaler.fit_transform(df_three[numerical_cols])

df_three.head()

,Z,X1,X2,X3,Y1,Y2,Y3,Noise1,Noise2
0,0.487759,1.056820,-0.028672,0,1.541848,1,0,-0.117110,1.038345
1,-0.161022,0.249294,-1.340393,0,1.349954,0,1,0.460532,0.029623
2,0.642015,0.576484,-0.125405,1,-0.598750,1,1,-0.811824,-0.473659
3,1.536382,1.059754,-0.392767,0,0.866472,1,1,0.166196,-0.245314
4,-0.258995,0.055735,0.788366,0,1.213642,1,1,-0.328335,-0.169848


In [7]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_three.columns)  # nombres de las variables
data_three = df_three.to_numpy()

graph, edges = fci(
        data_three,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('X1','X2')]
)

Depth=0, working on node 8: 100%|██████████| 9/9 [00:02<00:00,  4.18it/s]


Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA


In [12]:
np.random.seed(42)
N = 300
L = np.random.randn(N)
X1 = 2 * L + np.random.randn(N) * 0.2
X2 = -3 * L + np.random.randn(N) * 0.2
X3 = np.random.randn(N) * 5  # Ruido sin relación

df_latent3 = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3})



In [14]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_latent3.columns)  # nombres de las variables
data_latent = df_latent3.to_numpy()

graph, edges = fci(
        data_latent,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('X1','X2'), ('X1','X3'), ('X2','X3')]
)

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 256.61it/s]


Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X1-X2: CIRCLE - CIRCLE
Arista X1-X3: ELIMINADA
Arista X2-